In [0]:
dbutils.widgets.text("UNIFIED_SALES_TABLE_PATH", "")
dbutils.widgets.text("UC_CATALOG_NAME", "")
dbutils.widgets.text("UC_SCHEMA_NAME", "")


In [0]:
UNIFIED_SALES_TABLE_PATH = dbutils.widgets.get("UNIFIED_SALES_TABLE_PATH")
if not UNIFIED_SALES_TABLE_PATH:
    dbutils.notebook.exit("Please provide a path for the UnifiedSalesDeltaTable.")

UC_CATALOG_NAME = dbutils.widgets.get("UC_CATALOG_NAME")
if not UC_CATALOG_NAME:
    dbutils.notebook.exit("Please provide catalog name.")

UC_SCHEMA_NAME = dbutils.widgets.get("UC_SCHEMA_NAME")
if not UC_SCHEMA_NAME:
    dbutils.notebook.exit("Please provide schema name.")

In [0]:
from delta.tables import DeltaTable

if DeltaTable.isDeltaTable(spark, UNIFIED_SALES_TABLE_PATH):
    dbutils.notebook.exit(f"UnifiedSalesDeltaTable has already been set up (path: {UNIFIED_SALES_TABLE_PATH}). If you need to configure specific components, please do so manually.")

In [0]:
spark.sql(f"""
CREATE SCHEMA IF NOT EXISTS {UC_CATALOG_NAME}.{UC_SCHEMA_NAME}      
""")

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {UC_CATALOG_NAME}.{UC_SCHEMA_NAME}.UnifiedSalesDeltaTable 
(
  SaleID LONG NOT NULL,
  SaleDate DATE NOT NULL,
  Customer STRING NOT NULL,
  Product STRING NOT NULL,
  Units INT NOT NULL,
  UnitPrice DOUBLE NOT NULL,
  PaymentMethod STRING NOT NULL,
  CreatedAt TIMESTAMP NOT NULL,
  Region STRING NOT NULL,  
  ProcessingTime TIMESTAMP NOT NULL,
  SourcePath STRING NOT NULL
)
USING DELTA
PARTITIONED BY (SaleDate, Region)
LOCATION '{UNIFIED_SALES_TABLE_PATH}';
""")


In [0]:
spark.sql(f"""
ALTER TABLE {UC_CATALOG_NAME}.{UC_SCHEMA_NAME}.UnifiedSalesDeltaTable 
ADD CONSTRAINT chk_customer_not_empty CHECK (LENGTH(Customer) > 0)
""")

In [0]:
spark.sql(f"""
ALTER TABLE {UC_CATALOG_NAME}.{UC_SCHEMA_NAME}.UnifiedSalesDeltaTable 
ADD CONSTRAINT chk_product_not_empty CHECK (LENGTH(Product) > 0)
""")

In [0]:
spark.sql(f"""
ALTER TABLE {UC_CATALOG_NAME}.{UC_SCHEMA_NAME}.UnifiedSalesDeltaTable 
ADD CONSTRAINT chk_units_positive CHECK (Units >= 0)
""")

In [0]:
spark.sql(f"""
ALTER TABLE {UC_CATALOG_NAME}.{UC_SCHEMA_NAME}.UnifiedSalesDeltaTable 
ADD CONSTRAINT chk_unit_price_greater_then_zero CHECK (UnitPrice > 0)
""")

In [0]:
spark.sql(f"""
ALTER TABLE {UC_CATALOG_NAME}.{UC_SCHEMA_NAME}.UnifiedSalesDeltaTable 
ADD CONSTRAINT chk_payment_method_not_empty CHECK (LENGTH(PaymentMethod) > 0)
""")

In [0]:
spark.sql(f"""
ALTER TABLE {UC_CATALOG_NAME}.{UC_SCHEMA_NAME}.UnifiedSalesDeltaTable 
ADD CONSTRAINT chk_region_valid CHECK (Region IN ('north', 'south', 'west'))
""")

In [0]:
spark.sql(f"""
ALTER TABLE {UC_CATALOG_NAME}.{UC_SCHEMA_NAME}.UnifiedSalesDeltaTable 
ADD CONSTRAINT chk_source_path_not_empty CHECK (LENGTH(SourcePath) > 0)
""")

In [0]:
spark.sql(f"""
ALTER TABLE {UC_CATALOG_NAME}.{UC_SCHEMA_NAME}.UnifiedSalesDeltaTable 
SET TBLPROPERTIES (
  'delta.autoOptimize.optimizeWrite' = 'true',
  'delta.autoOptimize.autoCompact' = 'true'
)
""")

In [0]:
dbutils.notebook.exit("UnifiedSalesDeltaTable setup successful.")
